# Content
     
**Data Preprocessing**  
   - Settings
   - Loading Data
   - Glossary
   - Dealing with missing Values
   - Fixing Data Types
   - Dealing Bad Values
   - Feature Engieenier

In [1]:
### Data Analisy Libraries ###
import numpy as np
import pandas as pd

import sqlite3
import warnings
warnings.filterwarnings("ignore")

import os

# for dirname, _, filenames in os.walk('C:\\Users\Alysson\Documents\Projects\Hotel-Booking-Cancelation'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

## Settings

In [2]:
def jupyter_menu_settings():   
    
    """ Best looking changes. """
    
    display(HTML("""
            <style>

            h1 {
            background-color: #0E2031;
            color: white;
            padding: 15px 15px;
            text-align: center;
            font-family: Arial, Helvetica, sans-serif;
            border-radius:10px 10px;
            }

            h2 {
            background-color: #153656;
            color: white;
            padding: 10px 10px;
            text-align: center;
            font-family: Arial, Helvetica, sans-serif
            border-radius:10px 10px;
            }

            </style>            
    """))

def jupyter_table_settings():
    pd.set_option('display.expand_frame_repr', False )
    pd.set_option('display.max_columns', 30)
    pd.set_option('display.max_rows', 30)

In [3]:
# data_raw = pd.read_csv('C:\\Users\Alysson\Documents\Projects\Hotel-Booking-Cancelation\data\hotel_bookings.csv')
# data=data_raw.copy()

In [6]:
conn = sqlite3.connect("../data/hotels.sqlite")
data_raw = pd.read_sql_query("SELECT * from hotels", conn)
conn.close()
data_raw.shape

(141947, 32)

In [9]:
data=data_raw.copy()

In [10]:
#data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141947 entries, 0 to 141946
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           141947 non-null  object 
 1   is_canceled                     141947 non-null  int64  
 2   lead_time                       141947 non-null  int64  
 3   arrival_date_year               141947 non-null  int64  
 4   arrival_date_month              141947 non-null  object 
 5   arrival_date_week_number        141947 non-null  int64  
 6   arrival_date_day_of_month       141947 non-null  int64  
 7   stays_in_weekend_nights         141947 non-null  int64  
 8   stays_in_week_nights            141947 non-null  int64  
 9   adults                          141947 non-null  int64  
 10  children                        141939 non-null  float64
 11  babies                          141947 non-null  int64  
 12  meal            

## Dealing with missing values

In [11]:
data.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               8
babies                                 0
meal                                   0
country                              625
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              19555
company         

In [12]:
#replacing missing values
nan_replacements = {"children": 0,"country": "unknown", "agent": 0, "company": 0}
data = data.fillna(nan_replacements)

In [20]:
data.isnull().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
company                           0
days_in_waiting_list              0
customer_type                     0
adr                         

## Fixing Data Types

In [21]:
data['reservation_status_date'] = pd.to_datetime(data['reservation_status_date'], format='%Y/%m/%d')

In [22]:
data = data.astype({"children": int, "agent": int, "company": int})

## Dealing Bad Values

In [23]:
#removing zero guests possibility
no_guest = ((data['children'] == 0) & (data['adults'] == 0) & (data['babies'] == 0))
data = data[~no_guest]

In [24]:
#removing negative values for transactions possibility
negative_ADR = (data["adr"] < 0)
data = data[~negative_ADR]

In [25]:
#same meaning values according to dictionary source
data['meal'].replace("Undefined", "SC", inplace=True)

## Feature Engieener

In [26]:
#these new columns will help our analysis further on
data["adr_pp"] = data["adr"] / (data["adults"] + data["children"])
data["people"] = (data["adults"] + data["children"] + data["babies"])
data['kids'] = data['children'] + data['babies']
data['days_stay'] = data['stays_in_weekend_nights'] + data['stays_in_week_nights']

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141736 entries, 0 to 141946
Data columns (total 36 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   hotel                           141736 non-null  object        
 1   is_canceled                     141736 non-null  int64         
 2   lead_time                       141736 non-null  int64         
 3   arrival_date_year               141736 non-null  int64         
 4   arrival_date_month              141736 non-null  object        
 5   arrival_date_week_number        141736 non-null  int64         
 6   arrival_date_day_of_month       141736 non-null  int64         
 7   stays_in_weekend_nights         141736 non-null  int64         
 8   stays_in_week_nights            141736 non-null  int64         
 9   adults                          141736 non-null  int64         
 10  children                        141736 non-null  int32  